In [ ]:
!pip install prophet

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import optuna
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from xgboost import plot_importance

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

#plt.style.use('dark_background')

df_train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv', index_col='row_id')
df_test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv', index_col='row_id')

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

In [ ]:
df_train.head()

### Explore data

In [ ]:
sns.barplot(y='country', x='num_sold', data=df_train, hue='store');
plt.xlabel('Country')
plt.ylabel('Sold');

In [ ]:
sns.barplot(y='country', x='num_sold', data=df_train, hue='product');
plt.xlabel('Country')
plt.ylabel('Sold');

In [ ]:
df_temp = df_train[(df_train['country'] == 'Finland') & (df_train['store'] == 'KaggleMart') & (df_train['product'] == 'Kaggle Mug')]

In [ ]:
sns.lineplot(data=df_temp, x='date', y='num_sold');

In [ ]:
from prophet import Prophet

In [ ]:
df_temp = df_temp[['date', 'num_sold']].rename({'date': 'ds', 'num_sold': 'y'}, axis=1)

In [ ]:
m = Prophet(changepoint_prior_scale=0.05,
           holidays_prior_scale=20)
m.add_country_holidays(country_name='FI')
m.fit(df_temp)

In [ ]:
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')

In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head()

In [ ]:
fig = m.plot_components(forecast)

In [ ]:
from prophet.plot import add_changepoints_to_plot
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
countries = ['Finland', 'Norway', 'Sweden']
stores = ['KaggleMart', 'KaggleRama']
products = ['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker']
prophet_countries = {
    'Finland': 'FI',
    'Norway': 'NO',
    'Sweden': 'SE'
}

In [ ]:
y_pred = []
future = m.make_future_dataframe(periods=365)
for country in countries:
    for store in stores:
        for product in products:
            df_temp = df_train[(df_train['country'] == country) & 
                               (df_train['store'] == store) & 
                               (df_train['product'] == product)]
            df_temp = df_temp[['date', 'num_sold']].rename({'date': 'ds', 'num_sold': 'y'}, axis=1)
            m = Prophet()
            m.add_country_holidays(country_name=prophet_countries[country])
            m.fit(df_temp)
            forecast = m.predict(future)
            forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
            fig = m.plot(forecast)
            plt.show(fig)
            y_pred.extend(forecast['yhat'].tolist()[-365:])

In [ ]:
result = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

In [ ]:
y_temp = []
for i in range(365):
    for j in range(18):
        y_temp.append(y_pred[i + j * 365])

In [ ]:
result['num_sold'] = y_temp

In [ ]:
result.to_csv('result.csv', index=False)